# **AI Workshop: From Agents to Intelligent Search**

## - Led by Ayaan Ahmad

### A hands-on guide on building and programming intelligent agents to solve complex problems.

# AI Workshop - Lab 4: A* (A-Star) Search

This is our "intelligent" algorithm. A* uses a heuristic (an educated guess) 
to find the optimal path much more efficiently than BFS.

We will:
1. Load our standard maze.
2. Define the heuristic and the A* algorithm.
3. Measure its performance.
4. Append the results to `performance_results.csv`.
5. Visualize its focused search path.

## Install Libraries

In [ ]:
pip install Pillow

### Importing Libraries

In [1]:
from pyamaze import maze, agent, COLOR
import os
import glob
from PIL import Image
from queue import PriorityQueue
import time # To measure execution time
import pandas as pd # To handle CSV data
import matplotlib.pyplot as plt # For plotting
import seaborn as sns # For better visualizations


# Create a list to store performance results
performance_results = []

# Part 3: Informed Search (Finding a Path Intelligently)

Uninformed searches are exhaustive. Informed search algorithms use problem-specific knowledge, called a heuristic, to find solutions more efficiently. A heuristic is an "educated guess" or a rule of thumb to guide the search.

### 3.1 - A* Search

A* Search is one of the most popular and effective informed search algorithms. It combines the cost to reach a node with an estimated cost to get from that node to the goal.

``g(n)`` = The actual cost of the path from the start node to node n.

``h(n)`` = The heuristic; an estimated cost of the cheapest path from node n to the goal.

``f(n) = g(n) + h(n)`` = The estimated cost of the cheapest solution through n.

For our maze, we will use the Manhattan distance as our heuristic. It's the distance between two points measured along axes at right angles (like walking on city blocks).

In [2]:
# The Heuristic Function (Manhattan Distance)
def h(cell1, cell2):
    x1, y1 = cell1; x2, y2 = cell2
    return abs(x1 - x2) + abs(y1 - y2)

# The Heuristic Function (Manhattan Distance)
def h(cell1, cell2):
    x1, y1 = cell1; x2, y2 = cell2
    return abs(x1 - x2) + abs(y1 - y2)

def aStar(m):
    start = (m.rows, m.cols)
    goal = (1, 1)
    g_score = {cell: float('inf') for cell in m.grid}; g_score[start] = 0
    f_score = {cell: float('inf') for cell in m.grid}; f_score[start] = h(start, goal)
    pq = PriorityQueue(); pq.put((f_score[start], h(start, goal), start))
    aPath = {}
    explored_cells = [] # Added tracking for all processed cells

    while not pq.empty():
        currCell = pq.get()[2]
        explored_cells.append(currCell) # Record cell as it's processed
        
        # Goal check removed to allow for full exploration
        if currCell == goal: break
        
        for d in "ESNW":
            if m.maze_map[currCell][d]:
                if d == 'E': childCell = (currCell[0], currCell[1] + 1)
                elif d == 'W': childCell = (currCell[0], currCell[1] - 1)
                elif d == 'S': childCell = (currCell[0] + 1, currCell[1])
                elif d == 'N': childCell = (currCell[0] - 1, currCell[1])
                
                temp_g_score = g_score[currCell] + 1
                temp_f_score = temp_g_score + h(childCell, goal)
                
                if temp_f_score < f_score[childCell]:
                    g_score[childCell], f_score[childCell] = temp_g_score, temp_f_score
                    pq.put((temp_f_score, h(childCell, goal), childCell))
                    aPath[childCell] = currCell
                    
    # After full exploration, reconstruct the forward path
    fwdPath = {}
    cell = (1, 1)
    if cell in aPath or cell == start:
        while cell != start:
            fwdPath[aPath[cell]] = cell
            cell = aPath[cell]
            
    # Return the path and the complete list of explored cells
    return fwdPath, explored_cells

In [3]:
# --- Run A* ---
m = maze()
m.CreateMaze(loadMaze='workshop_maze.csv')

# --- Measure Execution Time ---
start_time = time.time()
final_path, explored_cells = aStar(m)
end_time = time.time()
astar_time = end_time - start_time
print(f"A* completed in {astar_time:.6f} seconds. Path length: {len(final_path)}. Explored cells: {len(explored_cells)}")

time_taken = end_time - start_time
explored_count = len(explored_cells)
path_length = len(final_path)

# Save Metrics to CSV
results_file = 'performance_results.csv'
new_data = {'Algorithm': 'A*', 'Time (s)': time_taken, 'Explored Cells': explored_count, 'Path Length': path_length}
if os.path.exists(results_file):
    df = pd.read_csv(results_file)
    df = df[df.Algorithm != 'A*']
    df = pd.concat([df, pd.DataFrame([new_data])], ignore_index=True)
else:
    df = pd.DataFrame([new_data])
df.to_csv(results_file, index=False)
print(f"A* performance data saved to '{results_file}'")

# --- 1. Demo the SEARCH path ---
explored_path = {explored_cells[i]: explored_cells[i+1] for i in range(len(explored_cells)-1)}
search_agent = agent(m, footprints=True, color=COLOR.cyan, filled=True, shape='square')
m.tracePath({search_agent: explored_path})

# --- 2. Demo the FINAL path ---
path_agent = agent(m, footprints=True, color=COLOR.blue)
m.tracePath({path_agent: final_path})

A* completed in 0.005546 seconds. Path length: 72. Explored cells: 318
A* performance data saved to 'performance_results.csv'


In [ ]:
m.run()

Observation: A* finds the same optimal path as BFS, but it explores a much smaller portion of the maze, making it far more efficient. It uses its "educated guess" to stay focused on paths that seem to be heading toward the goal.